In [14]:
train = pd.read_json('./raw_data/train.json')
test = pd.read_json('./raw_data/test.json')
val = pd.read_json('./raw_data/val_copy.json')

In [21]:
val_no_tag_songs = val[(val['tags'].apply(len) == 0) & (val['songs'].apply(len) == 0)]

# 유사도 측정할 테스트 문장

In [46]:
val_no_tag_songs = val_no_tag_songs[val_no_tag_songs['plylst_title'].apply(len) != 0]

In [45]:
val_nothing_df = val_no_tag_songs[val_no_tag_songs['plylst_title'].apply(len) == 0]

In [27]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
for i in api.analyze('안녕'):
    print(i)

안녕	안녕/MAG


In [89]:
for each in val_no_tag_songs['plylst_title'].iloc[:3]:
    print(each)

앨리스테이블
기분 좋은 재즈와 함께 만드는 달달한 하루
■■■■ 사랑,그리고이별 ■■■■


In [119]:
val_no_tag_songs['plylst_title'].iloc[3:7]

57        마쉬멜로우같은 멜로우한 음악
71     공부와 독서를 위한 #Newage
101          새벽에 감성 돋는 노래
115    인디, OST 음악을 만나다 50
Name: plylst_title, dtype: object

In [256]:
from khaiii import KhaiiiApi
api = KhaiiiApi()
ls = []
contents_tokens = []
for each in val_no_tag_songs['plylst_title'].iloc[:5]:
    for word in api.analyze(each):
        for morph in word.morphs:
            ls.append(morph.lex) 
    
    contents_tokens.append(ls)
    ls=[]

In [257]:
contents_tokens

[['앨리스테', '이블'],
 ['기분', '좋', '은', '재즈', '와', '함께', '만들', '는', '달', '달', '하', 'ㄴ', '하루'],
 ['■', '■■■', '사랑', ',', '그', '리', '고이별', '■■■■'],
 ['마쉬', '멜로', '우', '같', '은', '멜로우한', '음악'],
 ['공부', '와', '독서', '를', '위하', 'ㄴ', '#', 'Newage']]

In [238]:
# 1분
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Okt

t = Okt()
vectorizer = CountVectorizer(min_df = 1)

contents_tokens = [t.morphs(each) for each in train['plylst_title']]

In [ ]:
contents_for_vectorize = []

for content in contents_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + " " + word
    
    contents_for_vectorize.append(sentence)
    
len(contents_for_vectorize)

In [240]:
# 벡터라이즈 morphs
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')
X = vectorizer.fit_transform(contents_for_vectorize)
X

<115071x35576 sparse matrix of type '<class 'numpy.float64'>'
	with 514198 stored elements in Compressed Sparse Row format>

In [219]:
# 벡터라이즈 nouns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 1, decode_error = 'ignore')
X = vectorizer.fit_transform(contents_for_vectorize)
X

<115071x14477 sparse matrix of type '<class 'numpy.float64'>'
	with 292969 stored elements in Compressed Sparse Row format>

In [249]:
new_post = [val_no_tag_songs['plylst_title'].iloc[1234]]
new_post_tokens = [t.morphs(row) for row in new_post]

new_post_for_vectorize = []

for content in new_post_tokens:
    sentence = ''
    for word in content:
        sentence = sentence + ' ' + word
    
    new_post_for_vectorize.append(sentence)
    
new_post_for_vectorize

[' 어디서 익숙한 멜로디 가 ?! 명곡 을 샘플링 한 해외 음악 21']

In [250]:
new_post_vec = vectorizer.transform(new_post_for_vectorize)

In [222]:
def dist_norm(v1, v2):
    v1_normalized = v1 / sp.linalg.norm(v1.toarray())
    v2_normalized = v2 / sp.linalg.norm(v2.toarray())
    
    delta = v1_normalized - v2_normalized
    
    return sp.linalg.norm(delta.toarray())

In [223]:
import scipy as sp

def dist_raw(v1, v2):
    delta = v1 - v2
    return sp.linalg.norm(delta.toarray())

## 거리 구하고

In [251]:
dist = [dist_norm(each, new_post_vec) for each in X]

In [252]:
ls = []
for i in sorted(dist):
    ls.append(dist.index(i))

In [253]:
j = []
for i in train.iloc[ls].drop_duplicates('id')['tags']:
    j.append(i)
k = sum(j, [])[:11]
k

['재즈', '힙합', '랩', '클래식', '익숙한', '팝', '팝송', 'Pop', '광고음악', '션멘데스', '할시']

In [228]:
j = []
for i in train.iloc[ls].drop_duplicates('id')['tags']:
    j.append(i)
k = sum(j, [])[:11]
k

['힙합',
 '랩',
 'DJ',
 'Electronic',
 '디제잉',
 'Pop',
 'sampling',
 'EDM',
 '팝',
 'DJing',
 '힙합']